In [1]:
import pyspark
import findspark
findspark.init()
import os
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
findspark.find()

'C:\\spark\\spark-3.2.4-bin-hadoop2.7'

In [2]:
# spark session
sc = SparkContext('local', 'calif-coll-data-union')
spark = SparkSession(sc)

In [4]:
path = 'C:\\Users\\RoiMinuit\\Desktop\\data\\switrs_raw_csvs'

# find folders
folders = []
for folder in os.listdir(path):
    folder = os.path.join(path, folder)
    folders.append(folder)
    
# collect file name from each folder
f_names = []
for folder in folders:
    for f in os.listdir(folder):
        if f.endswith('CollisionRecords.txt'):
            f_names.append(os.path.join(folder, f))

In [5]:
# check shape of each df to ensure compatibility and store df
all_data = []
for df in f_names:
    df = spark.read.csv(df, header=True, inferSchema=True)
    all_data.append(df)

In [6]:
# combine dfs
if all_data:
    one_data = all_data[0]
    for df in all_data[1:]:
        one_data = one_data.union(df)

# check shape
# print(one_data.count(), len(one_data.columns))

In [11]:
one_data = one_data.coalesce(1)
output_path = "C:\\Users\\RoiMinuit\\Desktop\\data\\switrs_raw_csvs\\all_collision_data"
one_data.write.mode('overwrite').csv(output_path, header=True)
# manually rename the name as it has a long generic name starting with 'part-'

In [13]:
print(f"There are {one_data.count()} rows, and {len(one_data.columns)} columns in your final dataset.")

There are 33832687 rows, and 76 columns in your final dataset.


In [14]:
# close session when needed
sc.stop()